<a href="https://colab.research.google.com/github/skiesindisguise/ProjectCI-Kelompok16-PenerapanAlgoritmaABCUntukOptimalisasiPembentukanStudyGroup/blob/main/ImplementasiAlgoritmaABC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as nptesttest

In [ ]:
# Penentuan Constraint
DATA_FILEPATH = 'Dataset_Mahasiswa_Kehadiran_Aktivitas_IPK.xlsx'
NUM_STUDENTS = 500
NUM_GROUPS = 100  # (500 siswa / 100 kelompok = 5 siswa per kelompok)
NUM_CRITERIA = 5  # (Kehadiran, Diskusi, Tugas, E-Learning, IPK)
NUM_CATEGORIES = 3

# Parameter Algoritma ABC
POPULATION_SIZE = 50  # Jumlah "sumber makanan" (solusi)
MAX_ITERATIONS = 100
LIMIT = 20  # Batas trial sebelum Scout Bee dikirim

# Konfigurasi Fitness (Bobot Kriteria)
# bobot (prioritas) untuk 5 kriteria. Total = 1.0, IPK dan Tugas lebih penting
CRITERIA_WEIGHTS = np.array([
    0.1,  # Bobot Kehadiran
    0.2,  # Bobot Partisipasi Diskusi
    0.2,  # Bobot Nilai Tugas
    0.1,  # Bobot Nilai E-Learning
    0.4   # Bobot IPK
])

# total bobot adalah 1
assert np.isclose(np.sum(CRITERIA_WEIGHTS), 1.0), "Total bobot kriteria (CRITERIA_WEIGHTS) 1.0"

In [ ]:
# INISIALISASI DATA
# Bagian: Pengumpulan Data, Data Preprocessing, dan prasyarat untuk perumusan Fitness

def load_student_data(filepath):
    print(f"Memuat data dari {filepath}...")
    try:
        df = pd.read_excel(filepath)
    except FileNotFoundError:
        print(f"Error: File data '{filepath}' tidak ditemukan.")
        return None, None

    # Mengambil 500 data pertama
    df = df.head(NUM_STUDENTS)

    # Menangani format IPK "3,77" (koma) dari Kaggle
    if 'IPK' in df.columns and df['IPK'].dtype == 'object':
        df['IPK'] = df['IPK'].str.replace(',', '.').astype(float)

    # Mengambil nama siswa untuk referensi output nanti
    student_names = df['Nama'].tolist()

    # Mengambil 5 kriteria sebagai array numpy untuk perhitungan
    try:
        student_data_raw = df[[
            'Kehadiran (%)',
            'Partisipasi Diskusi (skor)',
            'Nilai Tugas (rata-rata)',
            'Aktivitas E-Learning (skor)',
            'IPK'
        ]].values
        return student_data_raw, student_names
    except KeyError as e:
        print(f"Error: Nama kolom di Excel tidak sesuai. Kolom yang hilang: {e}")
        return None, None


def preprocess_data_to_categories(student_data_raw):
    print("Melakukan preprocessing data (Numerik -> Kategori)...")
    categorical_data = np.zeros_like(student_data_raw, dtype=int)

    for k in range(NUM_CRITERIA):
        criteria_values = student_data_raw[:, k]

        # batas (threshold) untuk kategori
        low_threshold = np.percentile(criteria_values, 33.3)
        high_threshold = np.percentile(criteria_values, 66.6)

        # kategori berdasarkan threshold
        categorical_data[:, k] = np.where(criteria_values <= low_threshold, 1,  # 1 = Kurang
                                 np.where(criteria_values <= high_threshold, 2, 3)) # 2 = Sedang, 3 = Baik

    return categorical_data


def calculate_target_distribution(categorical_data):
    print("Menghitung target distribusi (t_i^(k))...")
    target_dist = np.zeros((NUM_CRITERIA, NUM_CATEGORIES))

    for k in range(NUM_CRITERIA):
        for i_category in range(1, NUM_CATEGORIES + 1):
            category_count = np.sum(categorical_data[:, k] == i_category)
            target_dist[k, i_category - 1] = category_count / NUM_STUDENTS

    return target_dist